In [1]:
import glob, copy, pywt
import scipy
import wfdb
from sklearn import svm
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from collections import Counter
from ecgdetectors import Detectors
from read_data import ECGSample, ECGDataset

In [2]:
dataset = ECGDataset()

100
101
102
103
104
105
106
107
108
109
111
112
113
114
115
116
117
118
119
121
122
123
124
200
201
202
203
205
207
208
209
210
212
213
214
215
217
219
220
221
222
223
228
230
231
232
233
234


In [3]:
train= [ 101, 106, 108, 109, 112, 114, 115, 116, 118, 119, 122, 124, 201, 203, 205, 207, 208, 209, 215, 220, 223, 230]
test = [100, 103, 105, 111, 113, 117, 121, 123, 200, 202, 210, 212, 213, 214, 219, 221, 222, 228, 231, 232, 233, 234]

In [18]:
train_x, train_y, test_x, test_y = dataset.get_train_test_sets(train, test)
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

100
(2271, 39) 2271
101
(1864, 39) 1864
102
(2186, 39) 2186
103
(2083, 39) 2083
104
(2227, 39) 2227
105
(2572, 39) 2572
106
(2027, 39) 2027
107
(2136, 39) 2136
108
(1762, 39) 1762
109
(2530, 39) 2530
111
(2124, 39) 2124
112
(2537, 39) 2537
113
(1794, 39) 1794
114
(1879, 39) 1879
115
(1952, 39) 1952
116
(2411, 39) 2411
117
(1534, 39) 1534
118
(2277, 39) 2277
119
(1987, 39) 1987
121
(1862, 39) 1862
122
(2474, 39) 2474
123
(1517, 39) 1517
124
(1618, 39) 1618
200
(2600, 39) 2600
201
(1963, 39) 1963
202
(2135, 39) 2135
203
(2979, 39) 2979
205
(2656, 39) 2656
207
(1859, 39) 1859
208
(2953, 39) 2953
209
(3005, 39) 3005
210
(2648, 39) 2648
212
(2747, 39) 2747
213
(3249, 39) 3249
214
(2260, 39) 2260
215
(3361, 39) 3361
217
(2208, 39) 2208
219
(2154, 39) 2154
220
(2046, 39) 2046
221
(2427, 39) 2427
222
(2482, 39) 2482
223
(2605, 39) 2605
228
(2053, 39) 2053
230
(2255, 39) 2255
231
(1570, 39) 1570
232
(1780, 39) 1780
233
(3077, 39) 3077
234
(2752, 39) 2752
(51000, 39)
(51000,)
(49691, 39)
(49691,

In [44]:
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier

# from sklearn.decomposition import PCA
# # Make an instance of the Model
# pca = PCA(.90)
# pca.fit(train_x)
# tr_features_scaled = pca.transform(train_x)
# test_features_scaled = pca.transform(test_x)

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
scaler = StandardScaler()
scaler.fit(train_x)
lda = LDA(n_components=5)
tr_features_scaled = lda.fit_transform(scaler.transform(train_x), train_y)
test_features_scaled = lda.transform(scaler.transform(test_x))

# tr_features_scaled = scaler.transform(train_x)
# test_features_scaled = scaler.transform(test_x)


# cls = GradientBoostingClassifier(n_estimators=2000)
# cls.fit(train_x, train_y)
# train_score = cls.score(tr_features_scaled, train_y)
# test_score = cls.score(test_features_scaled, test_y)
# print("Train Score for the ECG dataset is about: {}".format(train_score))
# print("Test Score for the ECG dataset is about: {}".format(test_score))

C:\Users\tyeze\AppData\Local\Continuum\anaconda3\envs\genenv\lib\site-packages\sklearn\discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(39, 5 - 1) = 4 components.
  ChangedBehaviorWarning)
C:\Users\tyeze\AppData\Local\Continuum\anaconda3\envs\genenv\lib\site-packages\sklearn\discriminant_analysis.py:469: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


In [45]:
sum(train_y[train_y==5])

40

In [39]:
class_weights = {}
for c in range(4):
    class_weights.update({c+1:len(train_y) / float(np.count_nonzero(train_y == c+1))})


svm_model = sklearn.svm.SVC(kernel='rbf', degree=3, gamma='auto', 
                    coef0=0.0, shrinking=True,  tol=0.001, 
                    cache_size=200, class_weight=class_weights, verbose=False, 
                    max_iter=-1, random_state=None)

svm_model.fit(tr_features_scaled, train_y) 

SVC(C=1.0, break_ties=False, cache_size=200,
    class_weight={1: 1.1131968393940717, 2: 52.25409836065574,
                  3: 13.463569165786694, 4: 123.18840579710145},
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma='auto',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [40]:
svm_model.score(tr_features_scaled, train_y) 

0.8891372549019608

In [41]:
svm_model.score(test_features_scaled, test_y) 

0.6734418707613049

In [43]:
from sklearn.externals import joblib
# now you can save it to a file
joblib.dump(svm_model, 'svm_1_lda4_scaler.pkl') 
# and later you can load it
#clf = joblib.load('filename.pkl')

['svm_1_lda4_scaler.pkl']